In [79]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

In [80]:
# lets create a multiclass data 

# hyperparameters
NUM_CLASSES = 4
NUM_FEATURES = 2
RANDOM_SEED = 42

X_blob, y_blob = make_blobs(n_samples=1000,
                            n_features=NUM_FEATURES,
                            centers=NUM_CLASSES,
                            cluster_std=1.5,
                            random_state=RANDOM_SEED)

# we know that scikit-learn works on Numpy, so will be these data

In [89]:
# we can also create an accuracy function to measure how right our model is
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    accuracy = (correct / len(y_true)) * 100
    return accuracy

In [90]:
# to work with pytorch we need to transform our data into tensors
X_blob = torch.from_numpy(X_blob).type(torch.float)
y_blob = torch.from_numpy(y_blob).type(torch.LongTensor) 

# split our data into training and test
X_blob_train, X_blob_test, y_blob_train, y_blob_test = train_test_split(X_blob,
                                                                        y_blob,
                                                                        test_size=0.2,
                                                                        random_state=RANDOM_SEED)

# lets visualize our data
plt.figure(figsize=(10, 7))
plt.scatter(X_blob[:, 0], X_blob[:, 1], c=y_blob, cmap=plt.cm.RdYlBu)

TypeError: expected np.ndarray (got Tensor)

In [91]:
# create a model class now
# make it device-agnostics
device = "cuda" if torch.cuda.is_available() else "cpu"

class BlobModel(nn.Module):
    def __init__(self, input_features, output_features, hidden_units=8):
        """ Initializes multi-class classification model.

        Args:
            input_features (int): Number of input features to the model
            output_features (int): Number of output features (classes)
            hidden units (int): Number of hidden units between layers (default=8)

        Returns:
            None

        Examples:
            >>> model = BlobModel(2, 4)

        """
        super().__init__()
        self.linear_layer_stack =  nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_features)
        )

    def forward(self, x):
        return self.linear_layer_stack(x) 


In [92]:
# Now lets create an instance model from the model class and send it to the device
model_0 = BlobModel(input_features=2,
                    output_features=4,
                    hidden_units=8).to(device)

model_0

BlobModel(
  (linear_layer_stack): Sequential(
    (0): Linear(in_features=2, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=4, bias=True)
  )
)

In [93]:
# Before creating a training/evaluation loop, we should create a loss function and an optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(),
                            lr=0.1)


In [94]:
# Note that we put our model on the device, so before starting the loop, we should put our data on device as well.
X_blob_test, y_blob_test = X_blob_test.to(device), y_blob_test.to(device)
X_blob_train, y_blob_train = X_blob_train.to(device), y_blob_train.to(device)

In [95]:
print(y_logits.shape)
print(y_blob_train.shape)

torch.Size([800, 4])
torch.Size([800])


In [97]:
torch.manual_seed = 42
torch.cuda.manual_seed = 42
epochs = 100

for epoch in range(epochs):
    ## Training:
    # Step 0: put the model on the training mode
    model_0.train()

    # Step 1: forward pass
    y_logits = model_0(X_blob_train) # raw outputs of our model
    y_preds = torch.softmax(y_logits, dim=1).argmax(dim=1)# prediction label

    # Step 2: calculate the loss and accuracy
    train_loss = loss_fn(y_logits, y_blob_train)
    train_acc = accuracy_fn(y_true=y_blob_train,
                            y_pred=y_preds)

    # Step 3: zero gradiant of optimizer
    optimizer.zero_grad()

    # Step 4: backward propagation
    train_loss.backward()

    # Step 5:
    optimizer.step()

    ## Testing:
    # Step 0: put the model on evaluation mode
    model_0.eval()
    with torch.inference_mode():
        # Step 1:
        test_logits = model_0(X_blob_test)
        test_preds = torch.softmax(test_logits, dim=1).argmax(dim=1)

        # Step 2: calculate the loss and accuracy
        test_loss = loss_fn(test_logits, y_blob_test)
        test_acc = accuracy_fn(y_true=y_blob_test,
                                y_pred=test_preds)


    # Step 3: Print out what is happening
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, | Train Accuracy: {train_acc:.2f}%, | Test Accucary: {test_acc:.2f}%")


    


Epoch: 0, | Train Accuracy: 27.75%, | Test Accucary: 29.00%
Epoch: 10, | Train Accuracy: 96.00%, | Test Accucary: 98.50%
Epoch: 20, | Train Accuracy: 99.12%, | Test Accucary: 99.50%
Epoch: 30, | Train Accuracy: 99.12%, | Test Accucary: 99.50%
Epoch: 40, | Train Accuracy: 99.12%, | Test Accucary: 99.50%
Epoch: 50, | Train Accuracy: 99.12%, | Test Accucary: 99.50%
Epoch: 60, | Train Accuracy: 99.12%, | Test Accucary: 99.50%
Epoch: 70, | Train Accuracy: 99.12%, | Test Accucary: 99.50%
Epoch: 80, | Train Accuracy: 99.25%, | Test Accucary: 99.50%
Epoch: 90, | Train Accuracy: 99.25%, | Test Accucary: 99.50%


In [ ]:
# we can find some accuracy matrix on torchmetrics library
# from torchmetrics import Accuracy ...